# 순환 신경망

이 노트북을 주피터 노트북 뷰어(nbviewer.jupyter.org)로 보거나 구글 코랩(colab.research.google.com)에서 실행할 수 있습니다.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://nbviewer.jupyter.org/github/rickiepark/nn-in-tf/blob/master/recurrent_network.ipynb"><img src="https://jupyter.org/assets/main-logo.svg" width="28" />주피터 노트북 뷰어로 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/nn-in-tf/blob/master/recurrent_network.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
</table>

## IMDB 데이터셋 적재

순환 신경망을 위한 예제로 IMDB 데이터셋을 사용합니다. 이 데이터셋은 IMDB 영화 사이트의 리뷰 텍스트를 모아 놓은 것입니다. 리뷰 텍스트가 긍정적인지(1) 부정적인지(0)를 판단하는 이진 분류 문제입니다.

IMDB 데이터셋도 텐서플로에서 직접 다운로드할 수 있습니다. 먼저 텐서플로의 케라스 모듈을 임포트합니다.

In [ ]:
from tensorflow import keras

`datasets.imdb.load_data()` 함수를 사용하여 텍스트 데이터를 훈련 세트와 테스트 세트로 적재하겠습니다. 이 함수의 `num_words` 매개변수에서 사용할 단어의 수를 지정할 수 있습니다. 전체 단어의 수는 8만개가 넘기 때문에 여기에서는 자주 등장하는 1,000개의 단어만 사용하겠습니다.

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=1000)

훈련 세트와 테스트 세트의 크기를 확인해 보겠습니다. 각각 25,000개의 샘플이 들어 있습니다.

In [ ]:
print(x_train.shape, x_test.shape)

텐서플로의 IMDB 데이터셋은 리뷰에 등장하는 단어를 고유한 정수로 변환해 놓았습니다. 훈련 샘플 하나를 확인해 보겠습니다.

In [ ]:
print(x_train[0])

샘플이 담고 있는 값은 고유한 단어를 나타내는 정숫값입니다.

정수를 텍스트로 바꾸어 실제 어떤 단어들로 리뷰가 이루어져 있는지 확인해 보겠습니다. 각 단어에 대응하는 정숫값의 목록을 `get_word_index()` 함수에서 받아 올 수 있습니다. 목록을 다운로드한 후 `'love'` 단어의 인덱스를 확인해 보죠

In [ ]:
word_to_index = keras.datasets.imdb.get_word_index()

word_to_index['love']

숫자로 채워진 샘플을 텍스트로 바꾸기 위해 `word_to_index` 딕셔너리를 거꾸로 만들어 숫자를 단어에 매핑한 `index_to_word`를 만들어 보죠.

In [ ]:
index_to_word = {}
for k, v in word_to_index.items():
    index_to_word[v] = k

매핑이 잘 되었는지 116번 인덱스의 단어를 출력해 보겠습니다.

In [ ]:
print(index_to_word[116])

매핑이 잘 된 것 같습니다. 이제 첫 번째 샘플을 숫자에서 텍스트로 변경해 보겠습니다. `x_train[0]` 리스트에서 원솔르 하나씩 꺼내서 `index_to_word` 딕셔너리의 키로 전달합니다.

In [ ]:
for i in x_train[0]:
    print(index_to_word[i], end=' ')

'movie'나 'film' 같은 단어를 보니 확실히 영화 리뷰로 볼 수 있습니다. 하지만 대부분의 특별한 의미가 없는 단어가 많이 들어 있습니다. 이런 데이터로 얼마나 예측 성능을 낼 수 있을지 시도해 보죠.

이진 분류이므로 타깃 데이터는 0 또는 1을 가진 1차원 배열입니다.

In [ ]:
print(y_train.shape)

처음 10개의 타깃 데이터를 확인해 보겠습니다.

In [ ]:
print(y_train[0:10])

분류 문제이므로 양성 클래스(긍정적인 리뷰)와 음성 클래스(부정적인 리뷰)의 갯수를 확인해 보겠습니다.

In [ ]:
import numpy as np

np.bincount(y_train)

## 데이터 전처리

리뷰 텍스트의 길이는 샘플마다 다릅니다. 예를 들어 첫 번째와 두 번째 샘플의 길이를 출력해 보겠습니다.

In [ ]:
print(len(x_train[0]), len(x_train[1]))

샘플의 길이를 동일하게 맞추는 가장 간단한 방법은 정해진 길이 이상은 버리는 것입니다. 만약 샘플의 길이가 정해진 길이보다 짧을 때는 합성곱 층에서처럼 0으로 패딩합니다.

텐서플로는 이런 작업을 위한 도구를 제공합니다. `pad_sequence()` 함수는 `maxlen` 매개변수에 지정된 크기로 모든 샘플의 길이를 동일하게 맞춥니다. 여기에서는 200개의 단어만 사용하도록 지정합니다.

In [ ]:
x_train_seq = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=200)
x_test_seq = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=200)

이제 훈련 세트와 테스트 세트의 크기를 다시 확인해 보겠습니다.

In [ ]:
print(x_train_seq.shape, x_test_seq.shape)

훈련 세트와 테스트 세트는 모두 25,000개의 샘플이 있고 길이가 200인 2차원 넘파이 배열이 되었습니다.

두 번째 샘플을 출력해서 어떻게 패딩되었는지 확인해 보죠.

In [ ]:
print(x_train_seq[1])

출력을 보면 샘플의 앞쪽으로 0이 패딩되었습니다. `pad_sequence()` 함수의 `padding` 매개변수의 기본값이 앞쪽에 패딩하는 `'pre'`입니다. `'post'`로 하면 샘플의 뒤쪽으로 패딩하여 길이를 맞춥니다.

앞쪽에 패딩하는 이유는 네트워크에 데이터가 하나씩 전달될 때 패딩된 부분이 마지막에 주입되는 것보다 먼저 주입되는 것이 학습에 유리하기 때문입니다.

## 모델 구성

패딩된 시퀀스 데이터가 준비되었으므로 모델을 만들어 훈련해 보겠습니다. 여기에서도 `Sequential` 클래스를 사용하여 층을 추가하겠습니다.

In [ ]:
model = keras.Sequential()

첫 번째 추가할 층은 `Embedding` 층입니다. 임베딩은 정수로 표현된 단어 데이터를 실수 벡터로 바꾸어 주는 역할을 합니다. 단어는 독립적인 특성이 아닙니다. 단어가 가진 의미는 문장에서 사용된 위치에 따라 상대적인 거리로 표현할 수 있습니다. 예를 들어 '자동차'와 '트럭' 사이의 거리는 '자동차'와 '비행기' 사이보다 가깝습니다.

거리를 표현하는 대표적인 방법은 벡터입니다. 임베딩 층은 훈련 데이터로부터 단어 사이의 거리를 학습하여 실수 벡터로 바꾸어 줍니다. 실수 벡터의 크기는 사용자가 지정할 수 있습니다. 여기에서는 64개의 벡터로 단어를 표현하겠습니다. 임베딩 층의 입력은 총 단어 갯수입니다. 이 예제에서 1,000개의 단어를 사용하므로 1,000으로 지정합니다. 마지막으로 `input_length` 매개변수에 입력될 시퀀스의 길이를 지정합니다.

In [ ]:
model.add(keras.layers.Embedding(1000, 64, input_length=200))

그다음 순환 층을 추가합니다. 순환 층은 입력과 곱해지는 가중치와 이전 타임 스텝의 출력과 곱해지는 가중치가 따로 있습니다. 여기에서는 16개의 순환 셀을 사용하겠습니다.

In [ ]:
model.add(keras.layers.SimpleRNN(16))

순환 층을 여러개 쌓을 수도 있지만 여기에서는 한 개만 사용했습니다. 양성(1) 또는 음성(0)을 분류하는 이진 분류 문제이므로 마지막 층에는 하나의 유닛을 가진 완전 연결 층을 놓습니다.

활성화 함수는 시그모이드 함수를 사용합니다. 시그모이드 함수는 입력값을 0~1 사이로 압축하기 때문에 확률로 이해하기 좋습니다.

In [ ]:
model.add(keras.layers.Dense(1, activation='sigmoid'))

`summary()` 매개변수를 사용하여 모델의 구조를 출력해 보죠.

In [ ]:
model.summary()

첫 번째 층의 출력은 200개의 단어가 64개의 실수 벡터로 표현됩니다. 이 벡터는 차례대로 다음 순환층으로 전달되며 이 층은 16개의 셀 출력을 만듭니다. 마지막 완전 연결 층은 최종 출력으로 한 개의 실수를 만듭니다.

각 층의 가중치를 확인해 보겠습니다. 이 예제는 총 1,000개의 단어를 사용합니다. 임베딩 층은 단어마다 64개의 실수 벡터를 만들기 때문에 가중치 크기는 $1000 \times 64 = 64,000$입니다.

순환 층에는 두 개의 가중치가 있습니다. 먼저 임베딩 층의 출력인 64개의 입력과 곱해지는 16개의 셀이 있습니다.

$64 \times 16 = 1024$

그리고 이전 타임 스텝의 셀 출력인 16개의 입력과 16개의 셀이 다시 곱해집니다.

$16 \times 16 = 256$

마지막으로 셀마다 하나씩 총 16개의 절편이 있습니다. 따라서 순환 층의 총 가중치는 $1024 + 256 + 16 = 1296$개 입니다.

출력층은 16개의 입력과 곱해지는 하나의 유닛이 있고 절편도 하나이므로 가중치 크기는 17입니다.

## 모델 훈련 및 테스트

아담 옵티마이저를 사용하고 훈련 과정에서 정확도를 기록하겠습니다. 이진 분류에 사용할 손실 함수는 `'binary_crossentropy'`입니다. 이 손실 함수를 로지스틱 손실 함수라고도 부릅니다.

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['acc'])

훈련 데이터와 타깃 데이터를 전달하고 5번의 에포크 동안 훈련합니다.

In [ ]:
history = model.fit(x_train_seq, y_train, epochs=5)

훈련 과정 동안 기록된 손실과 정확도를 그래프로 그려 보겠습니다.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['loss'])
plt.show()

In [ ]:
plt.plot(history.history['acc'])
plt.show()

`evaluate()` 메서드로 테스트 세트에 대한 손실과 정확도를 평가합니다. 이 메소드는 기본적으로 손실 점수를 반환하며 `compile()` 메서의 `metrics` 매개변수에 지정한 항목도 계산하여 반환합니다.

In [ ]:
model.evaluate(x_test_seq, y_test)

마지막으로 `predict_classes()` 메서드로 처음 다섯개의 테스트 데이터에 대한 예측을 비교해 보겠습니다.

In [ ]:
model.predict_classes(x_test_seq[0:5])

In [ ]:
y_test[0:5]